In [2]:
# run this to shorten the data import from the files
import os
cwd = os.path.dirname(os.getcwd())+'/'
path_data = os.path.join(os.path.dirname(os.getcwd()), 'datasets/')


In [3]:
def send_message(policy, state, message):
    print("USER : {}".format(message))
    new_state, response = respond(policy, state, message)
    print("BOT : {}".format(response))
    return new_state

In [4]:
def interpret(message):
    msg = message.lower()
    if 'order' in msg:
        return 'order'
    if 'kenyan' in msg or 'colombian' in msg:
        return 'specify_coffee'
    return 'none'

In [6]:
def respond(policy, state, message):
    (new_state, response) = policy[(state, interpret(message))]
    return new_state, response

In [7]:
# exercise 01

"""
Form filling

You'll often want your bot to guide users through a series of steps, such as when they're placing an order.

In this exercise, you'll begin building a bot that lets users order coffee. They can choose between two types: Colombian and Kenyan. If the user provides unexpected input, your bot will handle this differently depending on where they are in the flow.

Your job here is to identify the appropriate state and next state based on the intents and response messages provided. For example, if the intent is "order", then the state changes from INIT to CHOOSE_COFFEE.

A function send_message(policy, state, message) has already been defined for you. It takes the policy, the current state, and message as arguments, and returns the new state as a result. Additionally, an interpret(message) function, similar to the one Alan described in the video, has been pre-defined for you.
"""

# Instructions

"""

    Define three states: INIT with value 0, CHOOSE_COFFEE with value 1, and ORDERED with value 2.
    Create a dictionary called policy with tuples as keys and values. Each key is a tuple containing a state and an intent, and each value is a tuple containing the next state and the response message. The messages have been filled in for you. Your job is to fill in the states.
    Instantiate a variable state with the value INIT.
    For each of the messages, call the send_message() function, passing in the policy, state, and message.

"""

# solution

# Define the INIT state
INIT = 0

# Define the CHOOSE_COFFEE state
CHOOSE_COFFEE = 1

# Define the ORDERED state
ORDERED = 2

# Define the policy rules
policy = {
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
}

# Create the list of messages
messages = [
    "I'd like to become a professional dancer",
    "well then I'd like to order some coffee",
    "my favourite animal is a zebra",
    "kenyan"
]

# Call send_message() for each message
state = INIT
for message in messages:    
    state = send_message(policy, state, message)

#----------------------------------#

# Conclusion

"""
Congrats! You just built your first chatbot with a state machine
"""

USER : I'd like to become a professional dancer
BOT : I'm sorry - I'm not sure how to help you
USER : well then I'd like to order some coffee
BOT : ok, Colombian or Kenyan?
USER : my favourite animal is a zebra
BOT : I'm sorry - would you like Colombian or Kenyan?
USER : kenyan
BOT : perfect, the beans are on their way!


'\nCongrats! You just built your first chatbot with a state machine\n'

In [9]:
def send_message(state, message):
    print("USER : {}".format(message))
    new_state, response = respond(state, message)
    print("BOT : {}".format(response))
    return new_state

In [11]:
def respond(state, message):
    (new_state, response) = policy_rules[(state, interpret(message))]
    return new_state, response

In [ ]:
# exercise 02

"""
Asking contextual questions

Sometimes your users need some help! They will have questions and expect the bot to help them.

In this exercise, you'll allow users to ask the coffee bot to explain the steps to them. As in the previous exercise, the answer they get will depend on where they are in the flow.
"""

# Instructions

"""

    Add two rules to your policy_rules to handle the intent "ask_explanation" when in the states INIT or CHOOSE_COFFEE.
    Inside the send_messages() function, call the send_message() function with state and msg as arguments to define the new state. Then, hit 'Submit Answer' to send the messages and see the bot's responses.

"""

# solution

# Define the states
INIT=0 
CHOOSE_COFFEE=1
ORDERED=2

# Define the policy rules dictionary
policy_rules = {
    (INIT, "ask_explanation"): (INIT, "I'm a bot to help you order coffee beans"),
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "ask_explanation"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    for msg in messages:
        state = send_message(state,msg)

# Send the messages
send_messages([
    "what can you do for me?",
    "well then I'd like to order some coffee",
    "what do you mean by that?",
    "kenyan"
])

#----------------------------------#

# Conclusion

"""
Good work! Your bot can now modify its answers by considering context
"""

In [ ]:
# exercise 03

"""
Dealing with rejection

What happens if you make a suggestion to your user and they don't like it? Your bot will look really silly if it makes the same suggestion again right away.

Here, you're going to modify your respond() function so that it accepts and returns 4 arguments:

    The user message as an argument, and the bot response as the first return value.
    A dictionary params including the entities the user has specified.
    A prev_suggestions list. When passed to respond(), this should contain the suggestions made in the previous bot message. When returned by respond(), it should contain the current suggestions.
    An excluded list, which contains all of the results your user has already explicitly rejected.

Your function should add the previous suggestions to the excluded list whenever it receives a "deny" intent. It should also filter out excluded suggestions from the response.
"""

# Instructions

"""

    Define a respond() function with 4 arguments: message, params, prev_suggestions, and excluded.
    Interpret the message and store the result in parse_data.
    The value of the "intent" key of parse_data is itself a dictionary of key-value pairs. Assign parse_data["intent"]["name"] to intent, and parse_data["entities"] to entities.
    If the intent is "deny", use the .extend() method of the excluded list to add prev_suggestions to it.
    Initialize the empty params dictionary and empty suggestions and excluded lists. Then, hit 'Submit Answer' to send the messages to the bot.

"""

# solution

# Define respond()
def respond(message, params, prev_suggestions, excluded):
    # Interpret the message
    parse_data = interpret(message)
    # Extract the intent
    intent = parse_data['intent']['name']
    # Extract the entities
    entities = parse_data['entities']
    # Add the suggestion to the excluded list if intent is "deny"
    if intent == "deny":
        excluded.extend(prev_suggestions)
    # Fill the dictionary with entities	
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    # Find matching hotels
    results = [
        r 
        for r in find_hotels(params, excluded) 
        if r[0] not in excluded
    ]
    # Extract the suggestions
    names = [r[0] for r in results]
    n = min(len(results), 3)
    suggestions = names[:2]
    return responses[n].format(*names), params, suggestions, excluded

# Initialize the empty dictionary and lists
params, suggestions, excluded = {}, [], []

# Send the messages
for message in ["I want a mid range hotel", "no that doesn't work for me"]:
    print("USER: {}".format(message))
    response, params, suggestions, excluded = respond(message, params, suggestions, excluded)
    print("BOT: {}".format(response))

#----------------------------------#

# Conclusion

"""
Great! Your bot can now handle negative feedback gracefully.
"""

In [3]:
# exercise 04

"""
Pending actions I

You can really improve the user experience of your bot by asking the user simple yes or no follow-up questions. One easy way to handle these follow-ups is to define pending actions which get executed as soon as the user says "yes", and wiped if the user says "no".

In this exercise, you're going to define a policy() function which takes the intent as its sole argument and returns two values: The next action to take and a pending action. The policy function should return this pending action when a "yes" or "affirm" intent is returned and should wipe the pending actions if a "no" or "deny" intent is returned.

Here, the interpret(message) function has been defined for you such that if "yes" is in the message, "affirm" is returned, and if "no" is in the message, then "deny" is returned.
"""

# Instructions

"""

    Define a function called policy() which takes intent as its argument.
    If the intent is "affirm", return a "do_pending" action and None.
    If the intent is "deny", return a "Ok" action and None.

"""

# solution

# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

#----------------------------------#

# Conclusion

"""
Excellent work! With a policy() function defined, you can now incorporate it into a send_message() function.
"""

'\n\n'

In [4]:
# exercise 05

"""
Pending actions II

Having defined your policy() function, it's now time to write a send_message() function which takes both a pending action and a message as its arguments and leverages the policy() function to determine the bot's response.

Your policy(intent) function from the previous exercise has been pre-loaded.
"""

# Instructions

"""

    Define a function called send_message() which takes in two arguments: pending and message.
    Pass in the interpretation of message as an argument to policy() and unpack the result into the variables action and pending_action.
    If the action is "do_pending" and pending is not None, print the pending response. Else, print the action.
    Inside the definition of the send_messages() function, call your send_message() function with pending and msg as arguments. Then, hit 'Submit Answer' to send the messages and see the results.

"""

# solution

# Define send_message()
def send_message(pending, message):
    print("USER : {}".format(message))
    action, pending_action = policy(interpret(message))
    if action == 'do_pending' and pending is not None:
        print("BOT : {}".format(pending))
    else:
        print("BOT : {}".format(action))
    return pending_action
    
# Define send_messages()
def send_messages(messages):
    pending = None
    for msg in messages:
        pending = send_message(pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "ok yes please"
])

#----------------------------------#

# Conclusion

"""
Very well done! Your bot can now follow up on its own suggestions!
"""

'\n\n'

In [5]:
# exercise 06

"""
Pending state transitions

You'll often need to briefly deviate from the flow of a conversation, for example to authenticate a user, before returning to the topic of discussion.

In these cases, it's often simpler - and easier to debug - if you save some actions/states as pending rather than adding ever more complicated rules.

Here, you're going to define a policy_rules dictionary, where the keys are tuples of the current state and the received intent, and the values are tuples of the next state, the bot's response, and a state for which to set a pending transition.
"""

# Instructions

"""

    Complete the policy_rules dictionary by filling in the values:
        A user starts in the INIT state.
        If the user is in the INIT state and tries to place an order, you should ask for their number and create a pending transition to the AUTHED state.
        This is the only policy rule which creates a pending transition, so the others simply have a pending state value of None.
    The pending state has been added as the second argument of the send_message() function, which now returns the new state as well as the pending state. Call this send_message() function inside send_messages(), unpacking the output into the variables state and pending.
    Hit 'Submit Answer' to send the messages to the bot!

"""

# solution

# Define the states
INIT=0
AUTHED=1
CHOOSE_COFFEE=2
ORDERED=3

# Define the policy rules
policy_rules = {
    (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
    (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Colombian or Kenyan?", None),    
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-1234",
    "kenyan"
])

#----------------------------------#

# Conclusion

"""
Fantastic! You just built a bot that can handle some fairly complicated conversations!
"""

'\n\n'

In [6]:
# exercise 07

"""
Putting it all together I

It's time to put everything you've learned in the course together by combining the coffee ordering bot with the ELIZA rules from chapter 1.

To begin, you'll define a function called chitchat_response(), which calls the predefined function match_rule() from back in chapter 1. This returns a response if the message matched an ELIZA template, and otherwise, None.

The ELIZA rules are contained in a dictionary called eliza_rules.
"""

# Instructions

"""

    Define a chitchat_response() function which takes in a message argument.
    Call the match_rule() function with eliza_rules and message as arguments. Unpack the output into response and phrase.
    If the response is "default", return None.
    If "{0}" is in the response, replace the pronouns of the phrase using replace_pronouns(), and then include the phrase in the response by using .format() on response.

"""

# solution

# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, phrase = match_rule(eliza_rules, message)
    # Return none if response is "default"
    if response == "default":
        return None
    if '{0}' in response:
        # Replace the pronouns of phrase
        phrase = replace_pronouns(phrase)
        # Calculate the response
        response = response.format(phrase)
    return response

#----------------------------------#

# Conclusion

"""
Wow! You've put it all together and your bot can now interleave chit-chat and functional conversation.
"""

'\n\n'

In [7]:
# exercise 08

"""
Putting it all together II

With your chitchat_response(message) function defined, the next step is to define a send_message() function. This function should first call chitchat_response(message) and only use the coffee bot policy if there is no matching message.
"""

# Instructions

"""

    Define a send_message() function which takes in 3 arguments: state, pending, and message.
    Call chitchat_response(message), storing the result in response. If there is a response, print it and return the state along with None.
    Unpack the policy_rules dictionary into the variables new_state, response, and pending_state. To do this, pass in a tuple consisting of state and interpret(message).
    If pending is not none, extract the new states and response by using pending as the key of policy_rules.


"""

# solution

# Define send_message()
def send_message(state, pending, message):
    print("USER : {}".format(message))
    response = chitchat_response(message)
    if response is not None:
        print("BOT : {}".format(response))
        return state, None
    
    # Calculate the new_state, response, and pending_state
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "do you remember when I ordered 1000 kilos by accident?",
    "kenyan"
])  

#----------------------------------#

# Conclusion

"""
Very well done!
"""

'\n\n'

In [14]:

generated = {0.2: "i'm gonna punch lenny in the back of the been a to the on the man to the mother and the father to simpson the father to with the marge in the for the like the fame to the been to the for my bart the don't was in the like the for the father the father a was the father been a say the been to me the do it and the father been to go. i want to the boy i can the from a man to be the for the been a like the father to make my bart of the father",
 0.5: "i'm gonna punch lenny in the back of the kin't she change and i'm all better it and the was the fad a drivera it? what i want to did hey, he would you would in your bus who know is the like and this don't are for your this all for your manset the for it a man is on the see the will they want to know i'm are for one start of that and i got the better this is. it whoce and i don't are on the mater stop in the from a for the be your mileat",
 1.0: "i'm gonna punch lenny in the back of the to to macks how screath. firl done we wouldn't wil that kill. of this torshmobote since, i know i ord did, can give crika of sintenn prescoam.whover my me after may? there's right. that up. there's ruining isay.oh.solls.nan'h those off point chuncing car your anal medion.hey, are exallies a off while bea dolk of sure, hello, no in her, we'll rundems... i'm eventy taving me to too the letberngonce",
 1.2: "i'm gonna punch lenny in the back of the burear prespe-nakes, 'lisa to isn't that godios.and when be the bowniday' would lochs meine, mind crikvin' suhle ovotaci!..... hey, a poielyfd othe flancer, this in are rightplouten of of we doll hurrs, truelturone? rake inswaydan justy!we scrikent.ow.. by back hous, smadge, the lighel irely.yes, homer. wel'e esasmoy ryelalrs all wronencay...... nank. i wenth makedyk. come on help cerzind, now, n"}

In [15]:
def sample_text(seed, temperature):
    return generated[temperature]

In [16]:
# exercise 09

"""
Generating text with neural networks

In this final exercise of the course, you're going to generate text using a neural network trained on the scripts of every episode of The Simpsons. Specifically, you'll use a simplified version of the sample_text() function that Alan described in the video.

It takes in two arguments: seed and temperature. The seed argument is the initial sequence that the network uses to generate the subsequent text, while the temperature argument controls how risky the network is when generating text. At very low temperatures, it just repeats the most common combinations of letters, and at very high temperatures, it generates complete gibberish. In order to ensure fast runtimes, the network in this exercise will only work for a subset of temperature values.
(https://www.datacamp.com/community/tutorials/facebook-chatbot-python-deploy)
After you finish this exercise, be sure to check out this tutorial by Alan where he walks you through how to connect a chatbot to Facebook Messenger!
"""

# Instructions

"""

    Set the seed to be "i'm gonna punch lenny in the back of the".
    For each of the riskiness values [0.2, 0.5, 1.0, 1.2], call the sample_text() function with the arguments seed and temperature.

"""

# solution

# Feed the seed text into the neural network
seed = "i'm gonna punch lenny in the back of the"

# Iterate over the different temperature values
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print("\nGenerating text with riskiness : {}\n".format(temperature))
    # Call the sample_text function
    print(sample_text(seed, temperature))

#----------------------------------#

# Conclusion

"""
Very cool! You just generated some text with a neural network. Scroll through the output to see the text generated with different values of the temperature parameter. And congratulations on completing the course! If you're itching to continue learning about chatbots, remember to check out this tutorial by Alan on how to connect your bot to Facebook Messenger!
"""


Generating text with riskiness : 0.2

i'm gonna punch lenny in the back of the been a to the on the man to the mother and the father to simpson the father to with the marge in the for the like the fame to the been to the for my bart the don't was in the like the for the father the father a was the father been a say the been to me the do it and the father been to go. i want to the boy i can the from a man to be the for the been a like the father to make my bart of the father

Generating text with riskiness : 0.5

i'm gonna punch lenny in the back of the kin't she change and i'm all better it and the was the fad a drivera it? what i want to did hey, he would you would in your bus who know is the like and this don't are for your this all for your manset the for it a man is on the see the will they want to know i'm are for one start of that and i got the better this is. it whoce and i don't are on the mater stop in the from a for the be your mileat

Generating text with riskiness : 1.0

i

"\nVery cool! You just generated some text with a neural network. Scroll through the output to see the text generated with different values of the temperature parameter. And congratulations on completing the course! If you're itching to continue learning about chatbots, remember to check out this tutorial by Alan on how to connect your bot to Facebook Messenger!\n"